In [29]:
import numpy as np
import torch
import torch.nn as nn 
from torch.autograd import Variable
import pennylane as qml
import math 
from sympy import *


In [32]:
#assuming r=1,
def cartesian_to_spherical(x, y, z):
    theta = math.acos(z / 1)  
    phi = math.atan2(y, x)
    if phi < 0:
        phi += 2*np.pi
    return [theta, phi]


    
    

def fibonacci_sphere(n):

    points = []
    phi = math.pi * (math.sqrt(5.) - 1.)  # golden angle in radians

    for i in range(n):
        y = 1 - (i / float(n - 1)) * 2  # y goes from 1 to -1
        radius = math.sqrt(1 - y * y)  # radius at y

        theta = phi * i  # golden angle increment

        x = math.cos(theta) * radius
        z = math.sin(theta) * radius
        points.append((x, y, z))
        sph_points = [cartesian_to_spherical(x,y,z) for x,y,z in points]
    for i in sph_points:
        print(i)
    return sph_points

print(fibonacci_sphere(20))

[1.5707963267948966, 1.5707963267948966]
[1.8772398352612223, 1.923456466274096]
[0.9129244620159346, 1.5029306837044287]
[2.1880045929481837, 0.9954613564342298]
[1.428292654390687, 2.5168655402409255]
[1.0784514139262247, 0.5675029669579127]
[2.6855123413096815, 2.1507310688234167]
[0.5429885401462826, 2.6072121937920865]
[1.9163689856366029, 0.16861355694868221]
[1.9617037803765562, 3.0846360071570804]
[0.44063989142928317, 6.159471808486976]
[2.7999185039743866, 5.792493445015743]
[1.0658798321901566, 3.4469961691365496]
[1.3698304536524626, 5.897722417211437]
[2.375266480698212, 3.893510836299411]
[0.6290208023981475, 4.533338241557743]
[2.0600703695341984, 5.396222854656704]
[1.5961592835271339, 4.051957611441728]
[1.2503204145667102, 5.052445971355824]
[1.5707963267948966, 4.71238898038469]
[[1.5707963267948966, 1.5707963267948966], [1.8772398352612223, 1.923456466274096], [0.9129244620159346, 1.5029306837044287], [2.1880045929481837, 0.9954613564342298], [1.428292654390687, 2.5

In [ ]:
dtype = torch.cuda.DoubleTensor if torch.cuda.is_available() else torch.DoubleTensor
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class VariationalQuantumClassifierInterface:
    def __init__(
            self,
            num_of_input,
            num_of_output,
            num_of_wires,
            num_of_layers,
            var_Q_circuit,
            var_Q_bias,
            qdevice):

        self.var_Q_circuit = var_Q_circuit
        self.var_Q_bias = var_Q_bias
        self.num_of_input = num_of_input
        self.num_of_output = num_of_output
        self.num_of_wires = num_of_wires
        self.num_of_layers = num_of_layers

        self.qdevice = qdevice

        self.dev = qml.device(self.qdevice, wires = num_of_wires)


    def set_params(self, var_Q_circuit, var_Q_bias):
        self.var_Q_circuit = var_Q_circuit
        self.var_Q_bias = var_Q_bias

    def init_params(self):
        self.var_Q_circuit = Variable(torch.tensor(0.01 * np.random.randn(self.num_of_layers, self.num_of_wires, 3), device=device).type(dtype), requires_grad=True)
        return self.var_Q_circuit

    def _statepreparation(self, angles):

        """Encoding block of circuit given angles

        Args:
            a: feature vector of rad and rad_square => np.array([rad_X_0, rad_X_1, rad_square_X_0, rad_square_X_1])
        """
        # num_of_input determines the number of rotation needed.

        for i in range(self.num_of_input):
            qml.RY(angles[i,0], wires=i)
            qml.RZ(angles[i,1], wires=i)

    def _layer(self, W):
        """ Single layer of the variational classifier.

        Args:
            W (array[float]): 2-d array of variables for one layer

        """

        # Entanglement Layer

        for i in range(self.num_of_wires):
            qml.CNOT(wires=[i, (i + 1) % self.num_of_wires])

        # Rotation Layer
        for j in range(self.num_of_wires):
            qml.Rot(W[j, 0], W[j, 1], W[j, 2], wires=j)

    def circuit(self, angles):

        @qml.qnode(self.dev, interface='torch')
        def _circuit(var_Q_circuit, angles):
            """The circuit of the variational classifier."""
            self._statepreparation(angles)
            weights = var_Q_circuit

            for W in weights:
                self._layer(W)


            k = self.num_of_input-1
            return [qml.expval(qml.PauliZ(k))]

        return _circuit(self.var_Q_circuit, angles)



    def forward(self, angles):

        return self.circuit(angles)

In [ ]:
vqc = VariationalQuantumClassifierInterface(
            num_of_input =12,
            num_of_output =1,
            num_of_wires=12,
            num_of_layers=2,
            var_Q_circuit=None,
            var_Q_bias = None,
            qdevice = "default.qubit",
            hadamard_gate = False,
            more_entangle = False)

class VQCTorch(nn.Module):
    def __init__(self):
        super().__init__()

        self.q_params = nn.Parameter(0.01 * torch.randn(2, 12, 3))
    # n = number of amino acids, in_x in range: (0,n)
    def get_angles_atan(self, in_x, n):
        angles = fibonacci_sphere(n)
        return torch.stack([angles[item][0], angles[items[1]] for item in in_x])

    def forward(self, batch_item):
        vqc.var_Q_circuit = self.q_params
        output_batch = []

        for single_item in batch_item:
            angles = self.get_angles_atan(single_item)

            q_out_elem = vqc.forward(angles)

            clamp = 1e-9
            normalized_output = torch.clamp(torch.stack(q_out_elem), min=clamp)
            output_batch.append(normalized_output)

        outputs = torch.stack(output_batch).view(len(batch_item), 1) 

        return outputs